In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec  7 15:10:47 2019

@author: Parth
"""
import settings
import mysql.connector
import pandas as pd
import time
import itertools
import math
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import datetime
from IPython.display import clear_output
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import tweepy
import numpy as np
import re
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
consumer_key='rNrnFupaEqKt0eb7hjbdHKdWg'
consumer_secret= 'DTTMoQOrCBmngaXmOnFhrBjdjwtT54x0AbGvNwwuqyYNWwEvc7'
access_token='1002268050513575936-gGrQUmDiMyCxO2Y88lc3ojqNzbtLGm'
access_token_secret='G572YTe2S5TQTTaXhFvl1WyNopa8ilrkgWSlCXBZQwU4C'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)
TABLE_NAME='trump'


db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="parth123n@#*",
    database="TRUMPDB",
    charset = 'utf8'
)


timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=30)).strftime('%Y-%m-%d %H:%M:%S')
query = "SELECT id_str, text, created_at, polarity, user_location FROM {}" \
                 .format(TABLE_NAME)
df = pd.read_sql(query, con=db_connection)
# UTC for date time at default

#df['created_at'] = pd.to_datetime(df['created_at'])


# Clean and transform data to enable time series
result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
#result['created_at'] = pd.to_datetime(result['created_at']).apply(lambda x: x.strftime('%m-%d %H:%M'))
#result = df
result = result.rename(columns={"id_str": "Num of '{}' mentions".format(TABLE_NAME), "created_at":"Time in UTC"})
time_series = result["Time in UTC"][result['polarity']==0].reset_index(drop=True)
fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(TABLE_NAME), color='polarity')
fig.show()
time.sleep(60)

#fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(settings.TRACK_WORDS[0]), color='polarity')
#fig.show()
#time.sleep(60)
'''
df=df.set_index('created_at')

sns.set(rc={'figure.figsize':(11, 4)})

df['polarity'].plot(linewidth=0.7)
'''



content = ' '.join(df["text"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

y=fd['Frequency']
x=fd['Word']

index=np.arange(len(x))
plt.bar(index,y)
plt.xlabel('Words',fontsize=10)
plt.ylabel('Frequency',fontsize=10)
plt.xticks(index,x,fontsize=10,rotation=30)
plt.show()






















In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 12 09:57:24 2019

@author: Parth
"""
import psycopg2
import active
import pickle
import mysql.connector
import pandas as pd
import time
import itertools
import math
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import datetime
from IPython.display import clear_output
import plotly.offline as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import tweepy
import numpy as np
import re
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob

query_word='Hiranandani'
word=[query_word.lower()]
query = "SELECT id, username,tweet_text, created_at,location,polarity FROM {}".format(query_word)


try:     
    conn = psycopg2.connect(database=query_word, user = "postgres", password = "parth123n@#*", host = "127.0.0.1", port = "5432")
except:
    print("Create database first")

df = pd.read_sql(query, con=conn)




#
# Clean and transform data to enable time series
result = df.groupby([pd.Grouper(key='created_at', freq='1D'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()

#result = df.groupby([pd.Grouper(key='Time'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
#result['created_at'] = pd.to_datetime(result['created_at']).apply(lambda x: x.strftime('%m-%d %H:%M'))
#result = df
result = result.rename(columns={"id": "Num of '{}' mentions".format(query_word), "created_at":"Time in UTC"})
time_series = result["Time in UTC"][result['polarity']==0].reset_index(drop=True)
fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(query_word), color='polarity')
fig.show()
time.sleep(60)

#fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(settings.TRACK_WORDS[0]), color='polarity')
#fig.show()
#time.sleep(60)



content = ' '.join(df["tweet_text"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(10), columns = ["Word","Frequency"]).drop([0]).reindex()

y2=fd['Frequency']
x2=fd['Word']

x1=list(x2)
x1.remove('n')
x=pd.Series(x1)

y1=list(y2)
y3=y1[1:]
y=pd.Series(y3)

index=np.arange(len(x))
plt.bar(index,y)
plt.xlabel('Words',fontsize=10)
plt.ylabel('Frequency',fontsize=10)
plt.xticks(index,x,fontsize=10,rotation=30)
plt.show()


In [2]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')



<bound method NDFrame.head of             COUNTRY  GDP (BILLIONS) CODE
0       Afghanistan           21.71  AFG
1           Albania           13.40  ALB
2           Algeria          227.80  DZA
3    American Samoa            0.75  ASM
4           Andorra            4.80  AND
..              ...             ...  ...
217  Virgin Islands            5.08  VGB
218       West Bank            6.64  WBG
219           Yemen           45.45  YEM
220          Zambia           25.61  ZMB
221        Zimbabwe           13.74  ZWE

[222 rows x 3 columns]>
